# Classificação por Mínimos Quadrados

## Instalação de dependências


Execute a célula abaixo para instalar dependências. Não é necessário re-executar no mesmo runtime.

In [1]:
using Pkg
Pkg.add(["CSV", "DataFrames", "CategoricalArrays", "DataFrameMacros", "LsqFit"])

   Resolving package versions...


  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


## Bibliotecas

In [2]:
using CSV, DataFrames, Random, CategoricalArrays, DataFrameMacros, Statistics, LinearAlgebra
import Base: ImmutableDict
using Base.Iterators: drop

## Leitura e tratamento dos dados anotados

In [3]:
train = DataFrame(CSV.File("train.csv"));
train[1:10,:]

Row,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
,Int64,Int64,Int64,String,String7,Float64?,Int64,Int64,String31,Float64,String15?,String1?
1,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,missing,S
2,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
3,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.925,missing,S
4,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1,C123,S
5,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.05,missing,S
6,6,0,3,"Moran, Mr. James",male,missing,0,0,330877,8.4583,missing,Q
7,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
8,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.075,missing,S
9,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,missing,S


Aqui, descartamos colunas com informações individuais que ou provavelmente têm pouca correlação com o resultado da previsão ou trazem informações redundantes

In [4]:
Ids = select(train, :PassengerId)
discarded_cols = ["PassengerId", "Name", "Ticket", "Cabin"]
for d in discarded_cols
  select!(train, Not(d))
end

Arrendondamos as idades que vêm fracionarias por algum motivo

In [5]:
train.Age = round.(Union{Missing, Int}, train.Age);

In [6]:
train[1:10,:]

Row,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
,Int64,Int64,String7,Int64?,Int64,Int64,Float64,String1?
1,0,3,male,22,1,0,7.25,S
2,1,1,female,38,1,0,71.2833,C
3,1,3,female,26,0,0,7.925,S
4,1,1,female,35,1,0,53.1,S
5,0,3,male,35,0,0,8.05,S
6,0,3,male,missing,0,0,8.4583,Q
7,0,1,male,54,0,0,51.8625,S
8,0,3,male,2,3,1,21.075,S
9,1,3,female,27,0,2,11.1333,S


Nessa etapa, codificamos as variáveis não-numéricas através da atribuição de ordinais a cada valor único encontrado, e guardamos o codificador para que esse processo seja reprodutível no conjunto de testes.

Além disso, substituem-se os valores ausentes pela média dos valores conhecidos.

In [7]:
# Tipo do codificador
struct Encoder
  mappings::Dict{Union{Missing, String}, Dict{String, Int64}}
end

# Geração da função codificadora
function generate_encoder(data::DataFrame, cat_vars::Vector{String})
  encoder = Encoder(Dict())
  for cat in cat_vars
    lvls = levels(categorical(train[!, cat]))
    map = Dict()
    for (i, lvl) in enumerate(lvls)
      push!(map, lvl => i - 1)
    end
    push!(encoder.mappings, cat => map)
  end
  return encoder
end

# Codificação e tratamento dos valores faltantes
function encode_and_fill!(data::DataFrame, cat_vars::Vector{String}; encoder::Union{Missing, Encoder} = missing)
  if ismissing(encoder)
    encoder = generate_encoder(data, cat_vars)
  end
  for k in keys(encoder.mappings)
    data[!, k] = map(passmissing(s -> encoder.mappings[k][s]), data[!, k])
  end
  for k in eachcol(data)
    replacement = mean(skipmissing(k))
    replacement = nonmissingtype(eltype(k)) == Int ? round(replacement) : replacement
    replace!(k, missing => replacement)
  end
  disallowmissing!(data)
  return encoder
end


encode_and_fill! (generic function with 1 method)

In [8]:
encoder = encode_and_fill!(train, ["Sex", "Embarked"]);

In [9]:
train[1:10,:]

Row,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Int64
1,0,3,1,22,1,0,7.25,2
2,1,1,0,38,1,0,71.2833,0
3,1,3,0,26,0,0,7.925,2
4,1,1,0,35,1,0,53.1,2
5,0,3,1,35,0,0,8.05,2
6,0,3,1,30,0,0,8.4583,1
7,0,1,1,54,0,0,51.8625,2
8,0,3,1,2,3,1,21.075,2
9,1,3,0,27,0,2,11.1333,2


## Modelamento

Na fase de modelamento, definimos as funções utilitárias para representar os procedimentos de *feature engineering*, ajuste do modelo e predição das classes por esse modelo ajustado.

Com a função ```solve``` implementa-se a solução do problema de mínimos quadrados $Ax = b$ por meio da pseudoinversa de $A$ definida através de sua decomposição SVD reduzida.

In [10]:
function solve(A::Matrix{Float64}, b::Vector{Float64})
  F = svd(A)
  x = F.V * Diagonal(F.S .^(-1)) * F.U' * b
  return x
end

solve (generic function with 1 method)

Então, define-se um tipo ```Model``` que contém um dicionário no qual as chaves são os nomes das *features* originais do *DataFrame* e os valores são pares ordenados ($\theta$, $f: Float64 \mapsto Float64$) contendo a função $f$ a ser aplicada elemento-a-elemento na *feature* de nome correspondente para se obter uma nova *feature* e $\theta$, o coeficiente da respectiva nova *feature* na combinação linear realizada pelo modelo quando aplicado para prever a classificação de cada individuo.

Além desse dicionário, ```Model``` contém uma função classificadora que mapeia os valores reais quaisquer resultantes da combinação linear das *features* nos valores 0 e 1 que representam as classes. Esse classificador utiliza de um hiper-parâmetro que é obtido durante o ajuste do modelo ao conjunto de dados de treino e então é utilizado pelo classificador como valor limite entre as duas classes.

In [11]:
const Feature = Pair{Float64, Function}
const ImutDict = ImmutableDict{String, Vector{Feature}}
ImmutableDict(pairs::Vector{Pair{K, V}}) where {K <: Union{String, Symbol}, V <: Union{Float64, Vector{Feature}}} =
  foldl(
    (accumulated, pair) -> ImmutableDict(accumulated, pair),
    drop(pairs, 1),
    init = ImmutableDict(first(pairs))
  )

ImmutableDict

In [12]:
# Esse classificador utiliza 'm' como um valor limite entre as duas classes
function simple_classifier(x::Float64; α::Float64 = 0.5, kwargs...)
  return x < α ? 0 : 1
end

mutable struct Model
  features::ImutDict  # Dicionário que determina como trasformar as colunas originais e quais os coeficientes na combinação linear das colunas transformadas
  classifier::Function  # Função que separa os valores de saída da combinação linear entre as duas classes
  hyperparams::ImmutableDict{Symbol, Float64} # Hiperparâmetros do classificador
end

# Esse construtor serve para manter imutável a lista de nomes de colunas do modelo de acordo com o presente no Dataframe data
function Model(data::DataFrame; classifier::Function = simple_classifier)
  init = Vector{Pair{String, Vector{Feature}}}();
  for name in names(data)
    push!(init, name => Vector{Feature}());
  end
  d = ImmutableDict(init);
  kwarg = Base.kwarg_decl(methods(classifier)[1], Float64)[1]
  hyperparams = ImmutableDict(kwarg => 0.5)
  Model(d, classifier, hyperparams)
end

# Essa função modifica 'model' in-place, inserindo uma função 'f' que transforma a coluna 'name' e inicializando o coeficiente dessa feature com 0
function add_feature!(model::Model, name::String, f::Function)
  @assert typeof(f(0.0)) == Float64
  push!(model.features[name], 0.0 => f)
end

# Essa função transforma o DataFrame 'data' in-place, descartando suas colunas originais e inserindo as transformadas de acordo com 'model'
function transform!(model::Model, data::DataFrame)
  for (name, feats) in pairs(model.features)
    for (i, feat) in enumerate(feats)
      @transform! data name*"_$i" = feat.second({name})
      m = maximum(map( x -> abs(x), data[!, name*"_$i"]))
      @assert m != 0
      @transform! data name*"_$i" = {name*"_$i"}/m
    end
    select!(data, Not(name))
  end
end

# Gera uma predição na forma de Dataframe com base nos coeficientes guardados por 'model' e aplicando sua função classificadora
function predict(model::Model, data::DataFrame; y_name::String = "Survived")
  result = zeros(size(data)[1])
  data = Matrix(data)
  idx = 1
  for feats in values(model.features)
    for feat in feats
      result += feat.first*data[:, idx]
      idx += 1
    end
  end
  classifier = x -> model.classifier(x; model.hyperparams...)
  result = classifier.(result)
  prediction = DataFrame(y = result)
  select!(prediction, :y => y_name)
  return prediction
end

# Transforma as colunas e gera uma predição para 'data'
function transform_predict!(model::Model, data::DataFrame; y_name::String = "Survived")
  transform!(model, data)
  return predict(model, data, y_name = y_name)
end

# Obtém coeficientes para 'model' com base na solução do problema de mínimos quadrados x*θ = y
# e calcula o hiperparâmetro 'm' do classificador com base na média simples entre as médias das previsões para cada classe
function fit!(model::Model, x::DataFrame, y::DataFrame)
  x = Matrix(x)
  b = Vector{Float64}(y[!,1])
  survived = y[!,1] .== 1.0
  params = solve(x, b)
  prediction = (x*params)
  any_survivors = foldl(|, survived)
  any_non_survivors = !foldl(*, survived)
  m =  mean([any_survivors ? mean(prediction[survived]) : 0.5,
            any_non_survivors ? mean(prediction[.!survived]) : 0.5])
  
  model.hyperparams = ImmutableDict(:α => m)
  idx = 1
  for feats in values(model.features)
    for (i, feat) in enumerate(feats)
      feats[i] = params[idx] => feats[i].second
      idx += 1
    end
  end
end

# Transforma as colunas de 'x' e realiza o ajuste de 'model' para o problema x*θ = y
function transform_fit!(model::Model, x::DataFrame, y::DataFrame)
  transform!(model, x)
  return fit!(model, x, y)
end

transform_fit! (generic function with 1 method)

As funções `XX_features!` e `add_feature_listX!` a seguir adicionam *in-place* conjuntos específicos de funções transformadoras ao modelo, com uma correspondência fixa com cada coluna do DataFrame passado como argumento.

In [13]:
# adiciona funções polinomiais monomiais de grau 1 às features de 'model' até 'max_deg' para todas as colunas de 'data'
function polynomial_features!(model::Model, data::DataFrame, max_deg::Int)
  for name in names(data)
    for i in 1:max_deg
      add_feature!(model, name, x -> x^i)
    end
  end

end

# adiciona features da forma x -> 1/(|x|+1)^k
function rational_features!(model::Model, data::DataFrame, max_deg::Int)
  for name in names(data)
    for i in 1:max_deg
      add_feature!(model, name, x -> (abs(x)+1.0)^-i)
    end
  end

end

# adiciona features da forma x -> cos(a*x) e x -> sin(a*x)
function trigonometric_features!(model::Model, data::DataFrame, I::UnitRange{Int})
  for name in names(data)
    for a in I
      add_feature!(model, name, x -> cos(a*x))
      add_feature!(model, name, x -> sin(a*x))
    end
  end
end

# adiciona features da forma x -> exp(-a*x)
function exponential_features!(model::Model, data::DataFrame, I::UnitRange{Int})
  for name in names(data)
    for a in I
      add_feature!(model, name, x -> exp(-a*x))
    end
  end
end

# alerta de feiura! Aparentemente esse é o jeito mais facil de
# instanciar uma função que alterna entre -1 e 1 já que não
# há um método de obter o valor atual de um iterador sem especificar
# o estado inicial
cycle_var = -1
function cycler()
  global cycle_var = -cycle_var
  return cycle_var
end

# substitui todas as features presentes em 'model' por suas versões com sinais alternados
# i.e. faz com que as novas funções transformadoras sejam como as antigas mas multiplicando por -1 em elementos alternados das colunas
function alternate_features!(model::Model)
  for feats in values(model.features)
    for i in 1:length(feats)
      f = feats[i].second #essa
      feats[i] = feats[i].first => (x -> cycler()*f(x))
    end
  end
end


alternate_features! (generic function with 1 method)

Funções que adicionam listas de *features* de uma única vez

In [14]:
function add_feature_list1!(model::Model, data::DataFrame)
  polynomial_features!(model, data, 1)
end

function add_feature_list2!(model::Model, data::DataFrame)
  add_feature!(model, "Sex", x -> 1.0)
  polynomial_features!(model, data, 4)
end

function add_feature_list3!(model::Model, data::DataFrame)
  trigonometric_features!(model, data, 1:1)
  exponential_features!(model, data, 1:2)
end

function add_feature_list4!(model::Model, data::DataFrame)
    rational_features!(model, data, 3)
end

function add_feature_list5!(model::Model, data::DataFrame)
  add_feature_list1!(model, data)
  alternate_features!(model)
  add_feature_list1!(model, data)
end


add_feature_list5! (generic function with 1 method)

## Validação

Nessa parte, são colocadas as implementações de funções que automatizam o pré-processamento dos dados para testes e avaliam o desempenho de modelos sobre tais dados.

Para realizar uma forma de validação das previsões feitas pelo modelo, dividimos o conjunto de treino em outro conjunto menor de treino, selecionado aleatoriamente, e um conjunto de validação (testes).

A proporção entre os tamanhos de cada conjunto é especificada através da razão entre o tamanho do novo conjunto de treino e o tamanho do conjunto original, então são retornados os conjuntos com os dados das variáveis de previsão e da variável a ser prevista.

In [15]:
function train_test_split(data::DataFrame, train_ratio::Float64; y_col::String = "Survived")
  @assert 0. <= train_ratio <= 1.
  rows = collect(axes(data, 1))
  shuffle!(rows)
  train_selection = rows .<= size(data)[1]*train_ratio
  train_part = DataFrame(data[train_selection, :])
  test_part = DataFrame(data[.!train_selection, :])
  xtrain = select(train_part, Not(y_col))
  ytrain = select(train_part, y_col)
  xtest = select(test_part, Not(y_col))
  ytest = select(test_part, y_col)
  return xtrain, ytrain, xtest, ytest
end

train_test_split (generic function with 1 method)

A métrica de desempenho de um modelo para esse problema de classificação é a acurácia das previsões *i.e.* a taxa de acerto na classificação, então define-se `accuracy_score` para obter essa pontuação com base nos valores previstos por um modelo e nos valores de referência.

In [16]:
function accuracy_score(y_pred::DataFrame, y_true::DataFrame)
  total = size(y_pred)[1]
  right = foldl(+, (y_pred .== y_true)[:,1])
  return right/total
end

accuracy_score (generic function with 1 method)

Como deseja-se fazer a classificação sobre os dados de teste posteriormente sem que se tenha qualquer forma de acesso às respostas para esse conjunto, a escolha do modelo em particular dado um certo conjunto de *features* (ou funções transformadoras) é determinada iterativamente através de um método que, inicialmente, divide o conjunto de treino em um subconjunto de treino que é amostrado aleatoriamente do original e outro de validação, então ajusta o modelo a esse subconjunto de treino e mede a acurácia das previsões sobre o conjunto de validação.

Esse procedimento se repete em cada iteração e, ao final, é mantido o modelo ajustado ao subconjunto de treino tal que a acurácia sobre o conjunto de validação é a maior obtida.

In [17]:
# obtém o "melhor modelo" como descrito acima, com o conjunto de feature especificado por 'feature_adder', ao longo de 'n_its', cada uma na qual
# a razão treino:validação é dada por 'train_test_ratio':1-'train_test_ratio'
function select_best_model!(model::Model, train::DataFrame; train_test_ratio::Float64 = 0.8, feature_adder::Function = add_feature_list1!, n_its::Int = 50)
  xtrain, ytrain, xtest, ytest = train_test_split(train, train_test_ratio)
  feature_adder(model, xtrain)
  transform_fit!(model, xtrain, ytrain)
  pred = transform_predict!(model, xtest)
  best_x = xtrain
  best_y = ytrain
  best_acc = accuracy_score(pred, ytest)
  for i in 1:n_its
    xtrain, ytrain, xtest, ytest = train_test_split(train, train_test_ratio)
    transform_fit!(model, xtrain, ytrain)
    pred = transform_predict!(model, xtest)
    acc = accuracy_score(pred, ytest)
    if acc > best_acc
      best_acc = acc
      best_x = xtrain
      best_y = ytrain
    end
  end
  fit!(model, best_x, best_y)
end

select_best_model! (generic function with 1 method)

In [18]:
# repete o tratamento dos dados realizado sobre o conjunto de treino, mantendo o mesmo 'encoder'
function preprocess!(data::DataFrame, encoder::Encoder)
  discarded_cols = ["PassengerId", "Name", "Ticket", "Cabin"]
  for d in discarded_cols
    select!(data, Not(d))
  end
  data.Age = round.(Union{Missing, Int}, data.Age)
  encode_and_fill!(data, ["Sex", "Embarked"], encoder = encoder)
end

# faz o pré-processamento e a transforma 'test_data' de acordo com a lista de features criada por 'feature_adder'
function preprocess_and_transform!(test_data::DataFrame, encoder::Encoder; feature_adder::Function = add_feature_list1!)
  preprocess!(test_data, encoder)
  model = Model(test_data)
  feature_adder(model, test_data)
  transform!(model, test_data)
end

preprocess_and_transform! (generic function with 1 method)

Finalmente, para se avaliar cada conjunto de *features*, utiliza-se uma função que cria um modelo com uma lista de *features* especificada e então calcula a acurácia média da classificação ao longo de um número especificado de iterações, utilizando o modelo ajustado obtido por `select_best_model!` e o conjunto de teste do *Kaggle* para validação.

Calcular a acurácia média de `n_trials` execuções é necessário pois a função `select_best_model!` não é determinística, então há flutuações da acurácia.

In [19]:
# 'train' é o conjunto original de treino, 'test' é o conjunto para validação e não utilizado nos ajustes, 'reference' contém as verdadeiras classificações dos indivíduos de 'test'
# a lista de features a ser avaliada é especificada por 'feature_adder', enquanto é necessário passar como argumento o mesmo 'encoder' utilizado no preprocessamento de 'train'
function get_average_accuracy(train::DataFrame, test::DataFrame, reference::DataFrame, encoder::Encoder; n_trials::Int = 20, feature_adder::Function = add_feature_list1!)
  test = copy(test)
  preprocess_and_transform!(test, encoder, feature_adder = feature_adder)
  m = 0.0
  for i in 1:n_trials
    model = Model(select(train, Not(:Survived)))
    select_best_model!(model, train, n_its = 10, feature_adder = feature_adder, train_test_ratio = 0.8)
    prediction = predict(model, test)
    m += (accuracy_score(prediction, reference) - m)/i
  end
  m = m*100
  println("Acurácia média: $m %")
  return m
end

get_average_accuracy (generic function with 1 method)

A seguir, são mostradas as acurácias médias obtidas para cada lista de *features* definidas pelas funções `add_feature_listX!`.

In [20]:
test = DataFrame(CSV.File("test.csv"));
reference = select!(DataFrame(CSV.File("reference.csv")), :Survived);

In [21]:
get_average_accuracy(train, test, reference, encoder, n_trials = 30);

Acurácia média: 73.92344497607655 %


In [22]:
get_average_accuracy(train, test, reference, encoder, n_trials = 30, feature_adder = add_feature_list2!);

Acurácia média: 75.78947368421053 %


In [23]:
get_average_accuracy(train, test, reference, encoder, n_trials = 30, feature_adder = add_feature_list3!);

Acurácia média: 74.92822966507177 %


In [24]:
get_average_accuracy(train, test, reference, encoder, n_trials = 30, feature_adder = add_feature_list4!);

Acurácia média: 77.2328548644338 %


In [25]:
get_average_accuracy(train, test, reference, encoder, n_trials = 30, feature_adder = add_feature_list5!);

Acurácia média: 73.652312599681 %


Com isso, escolhe-se a lista de *features* de maior acurácia média, a especificada por `add_feature_list4!` que adiciona features da forma $x \mapsto \frac{1}{(|x|+1)^k},\;\forall k \in \{1,2,3\}$. Com base nessa lista de *features* é obtido um novo modelo ajustado, aplicando-se `select_best_model!` com um número maior de iterações para se obter um desempenho maior.

In [26]:
preprocess_and_transform!(test, encoder, feature_adder = add_feature_list4!)

In [27]:
model = Model(select(train, Not(:Survived)))
select_best_model!(model, train, n_its = 500, feature_adder = add_feature_list4!)

In [28]:
predictions = predict(model, test)
acc = accuracy_score(predictions, reference)*100
println("Acurácia do modelo: $acc %")

Acurácia do modelo: 77.03349282296651 %


Com isso, pode-se observar a matriz de confusão do modelo ajustado para se obter detalhes sobre seu desempenho.

Nesse caso, a linha 1 corresponde às classificações feitas pelo modelo para os indivíduos da classe '0' (não-sobreviventes) e linha 2 à classe '1' (sobreviventes).

In [29]:
function confusion_matrix(predictions::DataFrame, reference::DataFrame)
  predictions = Vector{Float64}(predictions[!, "Survived"])
  reference = Vector{Float64}(reference[!, "Survived"])
  survivors_predictions = predictions[reference .== 1.0]
  non_survivors_predictions = predictions[reference .== 0.0]
  confusion_matrix = Matrix{Int}(undef, (2, 2))
  confusion_matrix[1, 1] = length(non_survivors_predictions[non_survivors_predictions .== 0.0])
  confusion_matrix[1, 2] = length(non_survivors_predictions) - confusion_matrix[1, 1]
  confusion_matrix[2, 2] = length(survivors_predictions[survivors_predictions .== 1.0])
  confusion_matrix[2, 1] = length(survivors_predictions) - confusion_matrix[2, 2]
  return confusion_matrix
end

confusion_matrix (generic function with 1 method)

In [30]:
conf_mat = confusion_matrix(predictions, reference)

2×2 Matrix{Int64}:
 212   48
  48  110

In [31]:
function get_rates(conf_mat::Matrix{Int}, i::Int)
  @assert size(conf_mat) == (2, 2)
  @assert 1 <= i <= 2
  tpr = conf_mat[i, i]/(conf_mat[i, 1] + conf_mat[i, 2])
  ffr = 1 - tpr
  j = (3-i) % 3
  tfr = conf_mat[j, j]/(conf_mat[j, 1] + conf_mat[j, 2])
  fpr = 1 - tfr
  return tpr, fpr, tfr, ffr
end
tpr, fpr, tfr, ffr = get_rates(conf_mat, 2)
println("Taxa de verdadeiros positivos entre os sobreviventes: $(100*tpr) %")
println("Taxa de falsos positivos entre os sobreviventes: $(100*fpr) %")
println("Taxa de verdadeiros positivos entre os não-sobreviventes: $(100*tfr) %")
println("Taxa de falsos positivos entre os não-sobreviventes: $(100*ffr) %")

Taxa de verdadeiros positivos entre os sobreviventes: 69.62025316455697 %
Taxa de falsos positivos entre os sobreviventes: 18.461538461538463 %
Taxa de verdadeiros positivos entre os não-sobreviventes: 81.53846153846153 %
Taxa de falsos positivos entre os não-sobreviventes: 30.379746835443033 %


In [32]:
n_surv = foldl(+ , train[!,:Survived] .== 1.0)
n_nsurv = foldl(+ , train[!,:Survived] .== 0.0)
println("Número de sobreviventes no conjunto de treino: ", n_surv)
println("Número de não-sobreviventes no conjunto de treino: ", n_nsurv)
println("Diferença percentual Sobreviventes/Não-Sobreviventes: $((n_nsurv-n_surv)*100/n_surv) %")

Número de sobreviventes no conjunto de treino: 342
Número de não-sobreviventes no conjunto de treino: 549
Diferença percentual Sobreviventes/Não-Sobreviventes: 60.526315789473685 %


De fato, como a classe dos não-sobreviventes tem cerca de 60,5 % mais indivíduos do que a classe dos sobreviventes, há o que se chama de desbalanceamento entre elas, o que, nesse caso, resultou em um modelo com um viés para classificar como não-sobrevivente.

Consequentemente, a taxa de falsos positivos nessa classe majoritária é quase o dobro da taxa de falsos positivos da classe minoritária.

Para atenuação desse viés então, realiza-se uma busca exaustiva em um intervalo para obter o valor de hiperparâmetro $\alpha$ que maximiza a acurácia da classificação sobre o conjunto de treino.

In [33]:
# realiza busca em intervalo de valores em torno do valor original do hiperparametro obtido por fit!
function fit_and_search!(model::Model, x::DataFrame, y::DataFrame)
    fit!(model, x, y)
    k = collect(keys(model.hyperparams))[1]
    h0 = collect(values(model.hyperparams))[1]
    grid = h0-0.7:0.001:h0+0.7
    prediction = predict(model, x)
    best_acc = accuracy_score(prediction, y)
    best_h = h0
    for h in grid
        model.hyperparams = ImmutableDict(k => h);
        prediction = predict(model, x)
        acc = accuracy_score(prediction, y)
        if acc > best_acc
            best_acc = acc
            best_h = h
        end
    end
    model.hyperparams = ImmutableDict(k => h0);
end

# similar a select_best_model! mas com a busca pelo valor ótimo do hiperparâmetro no fit sobre o melhor conjunto de treino
function best_model_with_search!(model::Model, train::DataFrame; train_test_ratio::Float64 = 0.8, feature_adder::Function = add_feature_list1!, n_its::Int = 50)
    xtrain, ytrain, xtest, ytest = train_test_split(train, train_test_ratio)
    feature_adder(model, xtrain)
    transform_fit!(model, xtrain, ytrain)
    pred = transform_predict!(model, xtest)
    best_x = xtrain
    best_y = ytrain
    best_acc = accuracy_score(pred, ytest)
    for i in 1:n_its
        xtrain, ytrain, xtest, ytest = train_test_split(train, train_test_ratio)
        transform_fit!(model, xtrain, ytrain)
        pred = transform_predict!(model, xtest)
        acc = accuracy_score(pred, ytest)
        if acc > best_acc
        best_acc = acc
        best_x = xtrain
        best_y = ytrain
        end
    end
    fit_and_search!(model, best_x, best_y)
end

best_model_with_search! (generic function with 1 method)

In [34]:
model_no_search = Model(select(train, Not(:Survived)));
model_search = Model(select(train, Not(:Survived)));

In [35]:
select_best_model!(model_no_search, train, n_its = 800, feature_adder = add_feature_list4!)
println("Valor do hiperparâmetro α sem busca: $(model_no_search.hyperparams[:α])")

Valor do hiperparâmetro α sem busca: 0.4270472544065962


In [36]:
best_model_with_search!(model_search, train, n_its = 800, feature_adder = add_feature_list4!)
println("Valor do hiperparâmetro α com busca: $(model_search.hyperparams[:α])")

Valor do hiperparâmetro α com busca: 0.43647689411346924


A acurácia obtida com a busca no entanto é similar à obtida sem a busca.

In [37]:
predictions_no_search = predict(model_no_search, test)
predictions_search = predict(model_search, test)
acc_no_search = accuracy_score(predictions_no_search, reference)*100
acc_search = accuracy_score(predictions_search, reference)*100
println("Acurácia do modelo sem busca: $acc_no_search %")
println("Acurácia do modelo com busca: $acc_search %")

Acurácia do modelo sem busca: 77.75119617224881 %
Acurácia do modelo com busca: 77.75119617224881 %


Assim como as taxas de verdadeiros e falsos positivos.

In [38]:
conf_mat_search = confusion_matrix(predictions_search, reference)

2×2 Matrix{Int64}:
 216   44
  49  109

In [39]:
tpr, fpr, tfr, ffr = get_rates(conf_mat_search, 2)
println("Taxa de verdadeiros positivos entre os sobreviventes: $(100*tpr) %")
println("Taxa de falsos positivos entre os sobreviventes: $(100*fpr) %")
println("Taxa de verdadeiros positivos entre os não-sobreviventes: $(100*tfr) %")
println("Taxa de falsos positivos entre os não-sobreviventes: $(100*ffr) %")

Taxa de verdadeiros positivos entre os sobreviventes: 68.9873417721519 %
Taxa de falsos positivos entre os sobreviventes: 16.92307692307692 %
Taxa de verdadeiros positivos entre os não-sobreviventes: 83.07692307692308 %
Taxa de falsos positivos entre os não-sobreviventes: 31.0126582278481 %


Outra técnica para lidar com o desbalanceamento entre amostras de diferentes classe é a subamostragem da classe majoritária.

In [40]:
# retira um sobconjunto aleatório dos dados tal que os números de indivíduos em cada classe são iguais
function random_subsample(data::DataFrame, y_col::DataFrame)
  cl1_idx = Vector{Bool}((y_col .== 1.0)[!,1])
  cl0_idx = .! cl1_idx
  cl1_len = foldl(+, cl1_idx)
  cl0_len = foldl(+, cl0_idx)
  new_len = minimum([cl1_len, cl0_len])
  cl1_new = shuffle!(collect(1:cl1_len))[1:new_len]
  cl0_new = shuffle!(collect(1:cl0_len))[1:new_len]
  subset_idx = [cl1_new; cl0_new]
  data_new = data[subset_idx,:]
  y_col_new = y_col[subset_idx,:]
  return hcat(data_new, y_col_new)
end

random_subsample (generic function with 1 method)

In [41]:
x = select(train, Not(:Survived))
y = select(train, :Survived)
undersampled_train = random_subsample(x, y);

In [42]:
model = Model(x)
best_model_with_search!(model, undersampled_train, n_its = 800, feature_adder = add_feature_list4!, train_test_ratio = 0.85)

ImmutableDict{Symbol, Float64} with 1 entry:
  :α => 0.450491

No entanto, a aplicação dessa técnica aqui se mostrou pouco efetiva, causando um modelo com acurácia similar ou menor quando ajustado ao conjunto de treino subamostrado.
A ligeira diminuição da acurácia observada em parte das execuções pode ser atribuída à redução da informação relevante no conjunto subamostrado aleatóriamente, que não é compensada por qualquer redução significativa do viés do modelo, que mantém taxas similares de falsos positivos.

In [43]:
predictions = predict(model, test)
acc = accuracy_score(predictions, reference)*100
println("Acurácia do modelo: $acc %")

Acurácia do modelo: 77.99043062200957 %


In [44]:
conf_mat = confusion_matrix(predictions, reference)

2×2 Matrix{Int64}:
 218   42
  50  108

In [45]:
tpr, fpr, tfr, ffr = get_rates(conf_mat, 2)
println("Taxa de verdadeiros positivos entre os sobreviventes: $(100*tpr) %")
println("Taxa de falsos positivos entre os sobreviventes: $(100*fpr) %")
println("Taxa de verdadeiros positivos entre os não-sobreviventes: $(100*tfr) %")
println("Taxa de falsos positivos entre os não-sobreviventes: $(100*ffr) %")

Taxa de verdadeiros positivos entre os sobreviventes: 68.35443037974683 %
Taxa de falsos positivos entre os sobreviventes: 16.153846153846153 %
Taxa de verdadeiros positivos entre os não-sobreviventes: 83.84615384615385 %
Taxa de falsos positivos entre os não-sobreviventes: 31.645569620253166 %


Finalmente, realiza-se o ajuste com a implementação da biblioteca `LsqFit.jl` em vez do MMQ implementado, mas utilizando a mesma *feature engineering* escolhida como a melhor.

In [46]:
transform!(model, x);

In [47]:
using LsqFit
using LinearAlgebra: norm2

function lin_m(x,p)
  result = fill(0.0, size(x)[1])
  for i in 1:size(x)[2]
    result += p[i]*(x[:,i])
  end
  return result
end
p0 = fill(0.0, size(x)[2])
fit = curve_fit(lin_m, Matrix(x), Vector{Float64}(y[:,1]), p0);

In [48]:
function alpha_from_predictor(params::Vector{Float64}, x::Matrix{Float64}, y::Vector{Int64})
    survived = y .== 1
    prediction = (x*params)
    any_survivors = foldl(|, survived)
    any_non_survivors = !foldl(*, survived)
    alpha =  mean([any_survivors ? mean(prediction[survived]) : 0.5,
            any_non_survivors ? mean(prediction[.!survived]) : 0.5])
    return alpha
  end

alpha_from_predictor (generic function with 1 method)

In [49]:
alpha_lib = alpha_from_predictor(fit.param, Matrix(x), Vector(y[:,1]))

0.4327898014355756

In [50]:
classifier = x -> simple_classifier(x, α = alpha_lib)

#51 (generic function with 1 method)

In [51]:
predictions_lib = DataFrame(Survived = classifier.(Matrix(test)*fit.param))
acc_lib = accuracy_score(predictions_lib, reference)
println("Acurácia do modelo obtido com LsqFit.jl: $acc_lib %")

Acurácia do modelo obtido com LsqFit.jl: 0.7583732057416268 %


Pode ser observado que tanto acurácia quanto valor do hiperparâmetro $\alpha$ são similares aos obtidos com a implementação própria.